# Importing

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from sklearn_som.som import SOM
import cmocean.cm as cm
import salishsea_tools.viz_tools as sa_vi

# Dataset Inspection

In [23]:
ds = xr.open_dataset ('/results2/SalishSea/nowcast-green.202111/31jul22/SalishSea_1d_20220731_20220731_grid_T.nc')
ds

# Definition of variables
temp = ds.votemper 
saline = ds.vosaline

print(np.count_nonzero(temp[0,0]))
print(np.count_nonzero(np.isnan(temp[0,0])),'\n')

print(np.count_nonzero(saline[0,0]))
print(np.count_nonzero(np.isnan(saline[0,0])))



257641
176258 

257281
176258


In [3]:
# # Xarray implementation

# # First we stack (flatten) temp and saline, then we combine (concat) them in a single dataarray
# test = xr.concat([temp[0,0].stack(z=("x", "y")),saline[0,0].stack(z=("x", "y"))], dim = 'j')

# test = test.fillna(test)
# test=test.transpose()

# Data Wrangling

In [35]:
inputs = np.stack((temp[0,0].values.flatten(), saline[0,0].values.flatten()))

inputs2 = np.where(inputs==0, np.nan, inputs)
test = np.count_nonzero(inputs2[1])

indx = np.argwhere(~np.isnan(inputs2[1]))
indx3 = np.argwhere(~np.isnan(inputs2[0]) & np.isnan(inputs2[1]))

inputs2 = np.transpose(inputs2)

inputs3 = inputs2[~np.isnan(inputs2).any(axis=1)]

# Define the value for unwanted data (land and nans)




# SOM - Matrix Grid

In [5]:
temp_som = SOM(m=3, n=3, dim=2)
temp_som.fit(a)
predictions = temp_som.predict(a)

# Post processing 

unique_m, counts_m = np.unique(predictions, return_counts=True)

# Finding the unwanted cluster
unique_m[counts_m.argmax()]

predictions = np.where(predictions==unique_m[counts_m.argmax()], np.nan, predictions)
predictions = np.reshape(predictions,(898,398))
som_m = xr.DataArray(predictions,
                   coords = {'y': temp[0,0].y, 'x': temp[0,0].x},
                   dims = ['y','x'],
                   attrs=dict(description="Clusters of the performed self organizing map algorithm",
                              name ="Cluster",
                              units="count"),
                   )  

NameError: name 'a' is not defined

SOM - Vector Grid

In [ ]:
temp_som = SOM(m=9, n=1, dim=2)
temp_som.fit(a)
predictions = temp_som.predict(a)

# Post processing 

unique_v, counts_v = np.unique(predictions, return_counts=True)

# Finding the unwanted cluster
unique_v[counts_v.argmax()]

predictions = np.where(predictions==unique_v[counts_v.argmax()], np.nan, predictions)
predictions = np.reshape(predictions,(898,398))
som_v = xr.DataArray(predictions,
                   coords = {'y': temp[0,0].y, 'x': temp[0,0].x},
                   dims = ['y','x'],
                   attrs=dict(description="Clusters of the performed self organizing map algorithm",
                              name ="Cluster",
                              units="count"),
                   )  

In [ ]:
fig, ax = plt.subplots(ncols=2)

cmap = 'Accent'
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.8, 
                    hspace=0.4)

som_m.plot(ax=ax[0], cmap=cmap)
som_v.plot(ax=ax[1], cmap=cmap)
sa_vi.set_aspect(ax[0])
sa_vi.set_aspect(ax[1])
ax[0].title.set_text('Matrix Grid')
ax[1].title.set_text('Vector Grid')


